# AchromatCFW demo

This notebook demonstrates basic usage of the core functions in `achromatcfw.core.cfw`.

In [ ]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add ../src to the Python search path
sys.path.append(str(Path('..').resolve() / 'src'))

from achromatcfw.core.cfw import Edge, Farbsaum, Farbsaumbreite, ColorFringe

In [ ]:
# Load chromatic focal shift data
chl_path = Path('..') / 'data' / 'raw' / 'defocus_chl_zf85.csv'
chl_df = pd.read_csv(chl_path)
CHLdata = chl_df['defocus_um'].to_numpy(dtype=float)
CHLdata[:5]

In [ ]:
# Plot edge responses for each channel at z=0
xs = np.linspace(-50, 50, 200)
edge_R = [Edge('R', x, 0.0, CHLdata=CHLdata) for x in xs]
edge_G = [Edge('G', x, 0.0, CHLdata=CHLdata) for x in xs]
edge_B = [Edge('B', x, 0.0, CHLdata=CHLdata) for x in xs]

plt.figure(figsize=(6, 4))
plt.plot(xs, edge_R, label='R', color='r')
plt.plot(xs, edge_G, label='G', color='g')
plt.plot(xs, edge_B, label='B', color='b')
plt.xlabel('Pixel offset')
plt.ylabel('Edge response')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Compute fringe width as a function of defocus
z_vals = np.linspace(-50, 50, 25)
widths = [Farbsaumbreite(z, CHLdata=CHLdata) for z in z_vals]

plt.figure(figsize=(6, 4))
plt.plot(z_vals, widths, marker='o')
plt.xlabel('Defocus (z)')
plt.ylabel('Fringe width (pixels)')
plt.title('Fringe width vs. defocus')
plt.grid(True)
plt.show()

In [ ]:
# Inspect RGB responses for the central pixel
ColorFringe(0.0, 0.0, CHLdata=CHLdata)